<a href="https://colab.research.google.com/github/muskaanfayyaz/Banking-Agent-System/blob/main/Banking_Agent_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -Uq openai-agents
!pip install -Uq openai openai-agents

In [2]:
import importlib.util
print(importlib.util.find_spec("openai_agents"))

None


In [3]:
from agents import Agent, Runner
print("✅ openai_agents imported successfully!")


✅ openai_agents imported successfully!


In [4]:
import nest_asyncio
nest_asyncio.apply()
from google.colab import userdata

GEMINI_API_KEY = userdata.get("GEMINI_API_KEY")
if not GEMINI_API_KEY:
    raise ValueError("❌ No GEMINI_API_KEY found in Colab userdata. Go to Colab → Tools → Variables and set it.")


In [5]:
# =============================
# Banking Agent System
# =============================

# Imports from the Agents SDK
from agents import (
    Agent,
    Runner,
    RunContextWrapper,
    GuardrailFunctionOutput,
    input_guardrail,
    output_guardrail,
    RunConfig,
    AsyncOpenAI,
    OpenAIChatCompletionsModel,
    function_tool,
    set_tracing_disabled
)

# Standard libraries
from pydantic import BaseModel
import asyncio
import json

# Disable tracing
set_tracing_disabled(disabled=True)

# ✅ Use Colab-stored GEMINI_API_KEY (set in first cell)
if not GEMINI_API_KEY:
    raise ValueError("❌ No GEMINI_API_KEY found in Colab userdata.")

# Create async OpenAI-compatible Gemini client
client = AsyncOpenAI(
    api_key=GEMINI_API_KEY,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

# Define model wrapper
model = OpenAIChatCompletionsModel(
    model="gemini-2.0-flash",
    openai_client=client
)

# Global config
config = RunConfig(
    model=model,
    model_provider=client,
    tracing_disabled=True
)

# =============================
# Pydantic Models
# =============================
class UserInfo(BaseModel):
    name: str
    account_number: str
    balance: float
    pin: int

class InputCheck(BaseModel):
    is_bank_related: bool

# =============================
# Banking Agents
# =============================
deposit_agent = Agent(
    name="Deposit Agent",
    instructions="You are a deposit agent. Help users with deposits.",
    model=model,
)

withdrawal_agent = Agent(
    name="Withdrawal Agent",
    instructions="You are a withdrawal agent. Help users with withdrawals.",
    model=model,
)

balance_agent = Agent(
    name="Balance Agent",
    instructions="You are a balance agent. Call get_user_info if balance is requested.",
    model=model,
)

# =============================
# Guardrail Agents
# =============================
input_guardrail_agent = Agent(
    name="Guardrail Agent",
    instructions="""Return JSON: {"is_bank_related": true/false}.
    True if user asks about banking/accounts/payments; otherwise false.""",
    output_type=InputCheck,
    model=model,
)

@input_guardrail
async def banking_guardrail(
    ctx: RunContextWrapper[None], agent: Agent, user_input: str
) -> GuardrailFunctionOutput:
    """Blocks non-banking inputs."""
    res = await Runner.run(input_guardrail_agent, input=user_input, context=ctx.context, run_config=config)
    final = getattr(res, "final_output", None)
    is_bank = False

    if isinstance(final, dict):
        is_bank = bool(final.get("is_bank_related", False))
    elif hasattr(final, "is_bank_related"):
        is_bank = bool(getattr(final, "is_bank_related"))
    elif isinstance(final, str):
        try:
            parsed = json.loads(final)
            if isinstance(parsed, dict):
                is_bank = bool(parsed.get("is_bank_related", False))
        except Exception:
            is_bank = any(k in user_input.lower() for k in ["bank", "balance", "deposit", "withdrawal", "account"])

    return GuardrailFunctionOutput(output_info=user_input, tripwire_triggered=not is_bank)

output_guardrail_agent = Agent(
    name="Output Guardrail",
    instructions="Ensure response does not reveal sensitive info like PIN. Refuse if detected.",
    model=model,
)

@output_guardrail
async def output_guardrail_fn(
    ctx: RunContextWrapper[None], agent: Agent, output: str
) -> GuardrailFunctionOutput:
    """Filters sensitive info like PIN."""
    res = await Runner.run(output_guardrail_agent, input=output, context=ctx.context, run_config=config)
    final = getattr(res, "final_output", None)
    return GuardrailFunctionOutput(output_info=str(final or ""), tripwire_triggered=False)

# =============================
# Tool: User Info
# =============================
user_data = UserInfo(name="Shahid", account_number="42338734", balance=10000.0, pin=9876)

@function_tool
async def get_user_info(ctx: RunContextWrapper[None]) -> dict:
    """Returns user account details."""
    return {
        "user_name": user_data.name,
        "account_no": user_data.account_number,
        "response": f"Your current balance is ${user_data.balance:.2f}",
    }

# =============================
# Main Agent
# =============================
main_agent = Agent(
    name="Bank Agent",
    instructions="""You are a helpful bank agent:
    - Deposits -> handoff to Deposit Agent
    - Withdrawals -> handoff to Withdrawal Agent
    - Balance -> call get_user_info
    Return concise text or JSON {user_name, account_no, response}""",
    model=model,
    handoffs=[deposit_agent, withdrawal_agent, balance_agent],
    tools=[get_user_info],
    input_guardrails=[banking_guardrail],
    output_guardrails=[output_guardrail_fn],
)

# =============================
# Runner Example
# =============================
async def main():
    try:
        q1 = "what is my current balance?"
        q2 = "i have to deposit but i forget my pin, please help."

        print(f"\n🔹 Query 1: {q1}")
        r1 = await Runner.run(main_agent, input=q1, run_config=config)
        print("👉 Final Output:", r1.final_output)

        print(f"\n🔹 Query 2: {q2}")
        r2 = await Runner.run(main_agent, input=q2, run_config=config)
        print("👉 Final Output:", r2.final_output)

    except Exception as e:
        print(f"❌ Error: {e}")

# Run in Colab
await main()



🔹 Query 1: what is my current balance?
👉 Final Output: {'user_name': 'Shahid', 'account_no': '42338734', 'response': 'Your current balance is $10000.00'}


🔹 Query 2: i have to deposit but i forget my pin, please help.
👉 Final Output: I can certainly help you with your deposit, but since you've forgotten your PIN, we'll need to verify your identity first. Could you please provide the following information:

*   **Your full name:**
*   **Your account number:**
*   **Your date of birth:**
*   **The last four digits of your Social Security number:**

Once I have this information, I can help you reset your PIN or explore alternative deposit methods.

